In [17]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

In [18]:
!git clone https://github.com/coracai/homework.git

fatal: destination path 'homework' already exists and is not an empty directory.


In [19]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier

from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
%matplotlib inline


train = pd.read_csv('./homework/final/train_final.csv', engine='python')
test = pd.read_csv('./homework/final/test_final.csv', engine='python')



In [20]:
X_train = train.drop(["loan_status"], axis=1)
y_train = train["loan_status"]
feat_names = X_train.columns 

In [21]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

### LightGBM parameter tuning

In [ ]:
MAX_ROUNDS = 10000
def get_n_estimators(params , X_train , y_train, early_stopping_rounds = 20 ):
    lgb_params = params.copy()
    lgbtrain = lgb.Dataset(X_train , y_train )
    
    cv_result = lgb.cv(lgb_params , lgbtrain , num_boost_round=MAX_ROUNDS , nfold=3,  metrics ='binary_logloss', early_stopping_rounds=early_stopping_rounds, seed=3 )
     
    print('best n_estimators:' , len(cv_result['binary_logloss-mean']))
    print('best cv score:' , cv_result['binary_logloss-mean'][-1])
    
    return len(cv_result['binary_logloss-mean'])

params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'num_leaves': 60,
      'max_depth': 6,
      'bagging_fraction': 0.7,
      'feature_fraction': 0.7,
      'bagging_freq': 1,
      'extra_tress': False
      }

n_estimators_1 = get_n_estimators(params , X_train , y_train)

best n_estimators: 51
best cv score: 0.19831429221076116


In [ ]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'max_depth': 6,
      'bagging_freq': 1,
      'bagging_fraction': 0.7,
      'feature_fraction': 0.7,
      'extra_tress': False,
      'n_estimators': n_estimators_1
      }

lg = LGBMClassifier(silent=False,  **params)

num_leaves_s = range(50,90,10) #50,60,70,80
tuned_parameters = dict( num_leaves = num_leaves_s)

grid_search = GridSearchCV(lg, n_jobs=4, param_grid=tuned_parameters, cv = kfold, scoring="accuracy", verbose=5, refit = False)
grid_search.fit(X_train , y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   11.9s
[Parallel(n_jobs=4)]: Done  18 out of  20 | elapsed:   19.9s remaining:    2.2s


0.9201
{'num_leaves': 70}


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   20.2s finished


In [ ]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'max_depth': 6,
      'bagging_freq': 1,
      'bagging_fraction': 0.7,
      'feature_fraction': 0.7,
      'num_leaves': 70,
      'extra_tress': False,
      'n_estimators': n_estimators_1
      }

lg = LGBMClassifier(silent=False,  **params)

min_child_samples_s = range(10,50,10) 
tuned_parameters = dict( min_child_samples = min_child_samples_s)


grid_search = GridSearchCV(lg, n_jobs=4, param_grid=tuned_parameters, cv = kfold, scoring="accuracy", verbose=5, refit = False)
grid_search.fit(X_train , y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.1s


0.9201
{'min_child_samples': 20}


[Parallel(n_jobs=4)]: Done  18 out of  20 | elapsed:   20.3s remaining:    2.3s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   20.4s finished


In [ ]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'max_depth': 6,
      'bagging_freq': 1,
      'feature_fraction': 0.7,
      'num_leaves': 70,
      'extra_tress': False,
      'n_estimators': n_estimators_1,
      'min_child_samples': 20
      }

lg = LGBMClassifier(silent=False,  **params)

bagging_fraction_s = [i/10.0 for i in range(5,10)]
tuned_parameters = dict( bagging_fraction = bagging_fraction_s)


grid_search = GridSearchCV(lg, n_jobs=4, param_grid=tuned_parameters, cv = kfold, scoring="accuracy", verbose=5, refit = False)
grid_search.fit(X_train , y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.5s


0.9201
{'bagging_fraction': 0.7}


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   26.1s finished


In [ ]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'max_depth': 6,
      'bagging_freq': 1,
      'bagging_fraction': 0.7,
      'num_leaves': 70,
      'extra_tress': False,
      'n_estimators': n_estimators_1,
      'min_child_samples': 20
      }

lg = LGBMClassifier(silent=False,  **params)

feature_fraction_s = [i/10.0 for i in range(5,10)]
tuned_parameters = dict( feature_fraction = feature_fraction_s)


grid_search = GridSearchCV(lg, n_jobs=4, param_grid=tuned_parameters, cv = kfold, scoring="accuracy", verbose=5, refit = False)
grid_search.fit(X_train , y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.3s


0.9201
{'feature_fraction': 0.7}


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   25.6s finished


In [ ]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'learning_rate': 0.1,
      'max_depth': 6,
      'bagging_freq': 1,
      'bagging_fraction': 0.7,
      'feature_fraction': 0.7,
      'num_leaves': 70,
      'extra_tress': False,
      'min_child_samples': 20
      }
n_estimators_2 = get_n_estimators(params , X_train , y_train)

best n_estimators: 51
best cv score: 0.19812314486269358


In [22]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [70])
    metric: any = hp.choice('metric', ['binary_error'])
    n_estimators: any = hp.choice('n_estimators', [100])
    min_child_samples: any = hp.choice('min_child_samples', [20,30,40,50,60,70])
    max_depth: any = hp.choice('max_depth', [6])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.6, 0.8)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.6, 0.8)
    bagging_freq: any = hp.choice('bagging_freq', [1])
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt'])
    extra_trees: any = hp.choice('extra_tress', [False])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'boosting_type': 'gbdt', 'num_leaves': 70, 'metric': 'binary_error', 'objective': 'binary',
                'n_estimators': 100, 'max_depth': 6, 'bagging_freq': 1, 'min_child_samples': 20}


In [23]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [24]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        n_estimators = use_params.pop('n_estimators')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, n_estimators, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, n_estimators, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = n_estimators
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [25]:
fitter = LGBFitter(label='loan_status')

In [26]:
fitter.search_k_fold(kfold, train)

The minimum is attained in round 97
The minimum is attained in round 44
The minimum is attained in round 49
The minimum is attained in round 90
The minimum is attained in round 79
The minimum is attained in round 62
The minimum is attained in round 91
The minimum is attained in round 36
The minimum is attained in round 82
The minimum is attained in round 60
The minimum is attained in round 61
The minimum is attained in round 52
The minimum is attained in round 93
The minimum is attained in round 89
The minimum is attained in round 37
The minimum is attained in round 46
The minimum is attained in round 26
The minimum is attained in round 17
The minimum is attained in round 64
The minimum is attained in round 50
The minimum is attained in round 90
The minimum is attained in round 97
The minimum is attained in round 32
The minimum is attained in round 99
The minimum is attained in round 72
The minimum is attained in round 56
The minimum is attained in round 53
The minimum is attained in r

In [28]:
print(fitter.opt_params)

{'bagging_fraction': 0.6499835919662217, 'bagging_freq': 0, 'boosting': 0, 'device_type': 0, 'drop_rate': 0.11195490586366612, 'extra_tress': 0, 'feature_fraction': 0.7746845265430184, 'lambda_l1': 5.393883889100948, 'lambda_l2': 3.2664263739008983, 'learning_rate': 0.09140830604719294, 'max_depth': 0, 'metric': 0, 'min_child_samples': 0, 'min_gain_to_split': 0.6046839629397611, 'n_estimators': 0, 'num_leaves': 0, 'num_threads': 0, 'objective': 0, 'uniform_drop': 0}


In [29]:
params = {'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_threads': 4,
      'max_depth': 6,
      'bagging_freq': 1,
      'bagging_fraction': 0.6499835919662217,
      'feature_fraction': 0.7746845265430184, 
      'lambda_l1': 5.393883889100948, 
      'lambda_l2': 3.2664263739008983, 
      'learning_rate': 0.09140830604719294,
      'num_leaves': 70,
      'extra_tress': False,
      'min_child_samples': 30,
      'drop_rate': 0.11195490586366612,
      'n_estimators': 100,
      'min_gain_to_split': 0.6046839629397611,
      'metric': 'binary_error',
      'uniform_drop': True
      }

In [30]:
train_pred, test_pred, acc_result, models = fitter.train_k_fold(kfold, train, test, params = params)

The minimum is attained in round 65
The minimum is attained in round 35
The minimum is attained in round 25
The minimum is attained in round 76
The minimum is attained in round 24


In [31]:
acc_result

[0.07210000000000005,
 0.08109999999999995,
 0.08389999999999997,
 0.0827,
 0.08130000000000004]

In [32]:
from __future__ import division
convert_test_pred_df = pd.DataFrame((test_pred > 0.5).astype(int))
n=0
for i in range(len(convert_test_pred_df)):
  if convert_test_pred_df[0][i] == test['loan_status'][i]:
            n += 1
print(n)
print("Prediction accuracy: %f" % (n/len(convert_test_pred_df)) )

45902
Prediction accuracy: 0.918040
